## Likelihoods we have used:
1. regression: Gaussian
2. count regression: 
    Poisson: lambda=e^f,  Poisson2: lambda=ln(1+e^f)
3. binary classification: 
    Bernoulli: sigmoid,   Bernoulli2: Probit
    
For the sigmoid function, we use **expit** which is provided by *scipy.special*, because it is stable, fast and fairly accurate. Additionally, it is equivalent to sigmoid. For all methods, initial variational parameters are found by running the Laplace approximation on the subset/active set.

In [502]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.linalg import inv,norm,lstsq,cholesky
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
import scipy.io as sio
from sklearn import preprocessing
import random,time,GPy
from scipy.optimize import minimize
from scipy.special import loggamma,roots_hermitenorm,gamma,expit
from scipy.special import ndtr as std_norm_cdf

_sqrt_2pi = np.sqrt(2*np.pi)
_lim_val = np.finfo(np.float64).max
_lim_val_exp = np.log(_lim_val)

def std_norm_pdf(x): # define a standard normal pdf(from GPy)
    x = np.clip(x,-1e300,1e300)
    return np.exp(-np.square(x)/2)/_sqrt_2pi

def safe_exp(f):
    clip_f = np.clip(f, -np.inf, _lim_val_exp)
    return np.exp(clip_f)

def safe_ln(x, minval=0.0000000001):
    return np.log(x.clip(min=minval))

def rbf_kernel(X1, X2, l=1.0, sigma_f=1.0):
    '''
    Isotropic squared exponential kernel. Computes 
    a covariance matrix from points in X1 and X2.    
    Args:
        X1: Array of m points (m x d).
        X2: Array of n points (n x d).

    Returns:
        Covariance matrix (m x n).
    '''
    sqdist = np.sum(X1**2, 1).reshape(-1, 1) + np.sum(X2**2, 1) - 2 * np.dot(X1, X2.T)
    return sigma_f**2 * np.exp(-0.5 / l**2 * sqdist)

def kernel(X1, X2, *args, **kwargs):
    return rbf_kernel(X1, X2, *args, **kwargs)

def MFE(true_labels, pred_labels):
    '''
    Calculating mean fraction error(MFE) for count regression, the math is used as follows:
    MFE = mean( abs( (true_labels - pred_labels)./true_labels ) ) 
    Written by Kaikai
    '''    
    if true_labels.size != pred_labels.size:
        print('The size of true_labels and pred_labels is supposed to be identical.')
        return -1    
    true_labels = true_labels.flatten().astype('double'); pred_labels = pred_labels.flatten().astype('double')
    true_labels_temp = true_labels.copy()
    if 0 in true_labels: # replace zero with a very small positive value in order to avoid dividing by zero
        print('There are elements of zero value in true labels.')
        true_labels_temp[true_labels==0] = 1
    
    MFE = np.mean( np.abs( (true_labels - pred_labels)/true_labels_temp ) )
    return MFE

def calc_vlb(m,V, a, lik='Gaussian'):
    prior_mean_u = a[0]; prior_mean_f = a[1] # prior mean for inducing points    
    A = a[2] # Knm*inv(Kmm)
    Kmm = a[3]; Kmm_inv = a[4]; Kmn = a[5]; Knn_diag = a[6]; y = a[7] # the ground truth for training data
    noise_var = a[8]
    num_train = len(prior_mean_f);num_inducing = len(prior_mean_u)
    m_q = prior_mean_f + np.dot(A, (m-prior_mean_u)) # Eq.(3a) in paper
    v_q = ( Knn_diag.ravel() + np.diag(np.dot(A, np.dot(V-Kmm, A.T))) )[:, None] # Eq.(3b) in paper
    c1 = m - prior_mean_u; c2 = np.dot(Kmm_inv, c1)
    (Sign,LogDetKmm) = np.linalg.slogdet(Kmm); LogDetKmm = Sign*LogDetKmm
    (SignV,LogDetV) = np.linalg.slogdet(V); LogDetV = SignV*LogDetV;#print(v_q[:50])     
    if lik=='Bernoulli':
        f,w = GH_quad(m_q,np.sqrt(v_q));#vlb_lik = expit(y*f);print('yf',expit(y*f))
        vlb_lik = np.sum( 1/_sqrt_2pi*np.dot( safe_ln(expit(y*f)) ,w) ) # sigmoid liklihood
#         vlb_lik = np.sum( 1.0/np.sqrt(2*np.pi)*np.dot( np.log(std_norm_cdf(y*f)+1e-10) ,w) ) # Probit liklihood
    elif lik=='Gaussian':
        vlb_lik = -np.log(np.sqrt(2*np.pi*noise_var)) - np.sum((y-m_q)**2+v_q)/(2*noise_var)
    elif lik=='Poisson':
        vlb_lik = np.dot(y.T,m_q) - np.sum(loggamma(y+1)) - np.sum(np.exp(m_q+0.5*v_q))
    elif lik=='Poisson2': # another link func: lambda=ln(1+e^f)
        f,w = GH_quad(m_q,np.sqrt(v_q));
        term1 = -np.sum(loggamma(y+1)); term2 = -np.sum( 1/_sqrt_2pi*np.dot( np.log(1+safe_exp(f)),w) )
        term3 = np.sum( 1/_sqrt_2pi*np.dot( y*safe_ln(np.log(1+safe_exp(f))),w) )
        vlb_lik = term1 + term2 + term3
    vlb_kl = 0.5*( LogDetKmm - LogDetV + np.dot(c1.T, c2) + np.trace(np.dot(Kmm_inv,V)) - len(prior_mean_u) )
    vlb = vlb_lik - vlb_kl
    return vlb 

def get_init_hyperparameters(Z,Z_label,lik='Poisson'):
    # For all methods, initial variational parameters are found by running the Laplace approximation on the subset/active set.
    dim_data = Z.shape[1]
    kern = GPy.kern.RBF(dim_data, variance=1.0, lengthscale=1.0)
    likelihood = {
          'Poisson': GPy.likelihoods.Poisson(),
          'Poisson2': GPy.likelihoods.Poisson(GPy.likelihoods.link_functions.Log_ex_1()),
          'Gaussian': GPy.likelihoods.Gaussian(),
          'Bernoulli': GPy.likelihoods.Bernoulli()
        }[lik]
    laplace_inf = GPy.inference.latent_function_inference.Laplace()
    model_lap = GPy.core.GP(X=Z, Y=Z_label, likelihood=likelihood, inference_method=laplace_inf, kernel=kern)
    model_lap.optimize()
#     print(model_lap)
    return model_lap

def Adam(theta, g_t, t, alpha=0.01, m_t=0, v_t=0, opt='minimize'):
    beta_1 = 0.9; beta_2 = 0.999; epsilon = 1e-8     #initialize the values of the parameters    
    m_t = beta_1*m_t + (1-beta_1)*g_t                #updates the moving averages of the gradient
    v_t = beta_2*v_t + (1-beta_2)*(g_t*g_t)          #updates the moving averages of the squared gradient
    m_cap = m_t/(1-(beta_1**t))                      #calculates the bias-corrected estimates
    v_cap = v_t/(1-(beta_2**t))                      #calculates the bias-corrected estimates
    if opt=='maximize':
        theta = theta + (alpha*m_cap)/(np.sqrt(v_cap)+epsilon)    #updates the parameters
    else:
        theta = theta - (alpha*m_cap)/(np.sqrt(v_cap)+epsilon)    #updates the parameters
    return theta, m_t, v_t

In [496]:
def GH_quad(mu_star,std_star): # gauss_hermite_quad to calculate numerical integration, w and z denote weights and sample points, respectively. 
    z,w = roots_hermitenorm(n=50, mu=False); z,w=z[:,None],w[:,None]
    z = np.kron(std_star,z.T) + mu_star
    return z,w

def predict(X, xStar, l, sigma2, m, V,Kmm,Kmm_inv,y_min=4,y_max=20,noise_var=0.1,lik='Poisson'):
    num_xStar = xStar.shape[0]; Kmn = kernel(X, xStar, l, np.sqrt(sigma2))
    Knn_diag = np.eye(num_xStar)*sigma2+noise_var*np.eye(num_xStar)
    A = np.dot(Kmn.T, Kmm_inv)
    mu_star = np.dot(A,m); v2_star = Knn_diag + np.dot(A, np.dot(V-Kmm,A.T))
    std_star = np.sqrt(np.diag(v2_star))[:,None]
    if lik=='Bernoulli':
        return calc_Bernoulli_pred(mu_star,std_star)
    elif lik=='Gaussian':
        return calc_Gauss_pred(mu_star,std_star,noise_var)
    else: # Poisson, Poisson2
        return calc_Poisson_pred(mu_star,std_star,num_xStar,y_min,y_max,lik)

def calc_Gauss_pred(mu_star,std_star,noise_var):
    return mu_star, std_star+np.sqrt(noise_var)

def calc_Poisson_pred(mu_star,std_star,num_xStar,y_min,y_max,lik='Poisson'):
    f,w = GH_quad(mu_star,std_star)
    y_range = np.arange(y_min,y_max+1)
    lik_func = {
        'Poisson': lambda f,y: 1/gamma(y+1)*safe_exp(-safe_exp(f)+f*y),
        'Poisson2': lambda f,y: 1/gamma(y+1)*expit(-f)*( safe_ln(1.0+safe_exp(f))**y )
    }[lik]
    poisson_lik = np.zeros((num_xStar,len(y_range)))
    for i,y in enumerate(y_range):
        poisson_lik[:,i] = 1/_sqrt_2pi*np.dot(lik_func(f,y),w).ravel()
    return poisson_lik

def calc_Bernoulli_pred(mu_star,std_star):
#     v = std_star**2; kappa_v = (1+np.pi*v/8.0)**(-1/2) # Probit liklihood
#     p = sigmoid(kappa_v*mu_star) # p(y=1|x_*,m,V)
    f,w = GH_quad(mu_star,std_star) # sigmoid liklihood
    p = 1/np.sqrt(2*np.pi)*np.dot( expit(f), w ) # p(y=1|x_*,m,V)
    return p

def calc_m_q(m, A, prior_mean_u, prior_mean_f):
    return prior_mean_f + np.dot(A, (m-prior_mean_u))

def calc_v_q(V, A, Kmm, Knn_diag):
    return ( Knn_diag.ravel() + np.diag(np.dot(A, np.dot(V-Kmm, A.T))) )[:,None] # Eq.(3b) in paper

def calc_rho(m_q, v_q, y, lik='Poisson', noise_var=0.1):
    if lik=='Bernoulli':
        f,w = GH_quad(m_q,np.sqrt(v_q))
        return 1.0/_sqrt_2pi*np.dot(  y*expit(-y*f) , w ) # sigmoid liklihood
#         return 1.0/_sqrt_2pi*np.dot( y*std_norm_pdf(f) / (std_norm_cdf(y*f)+1e-10), w ) # Probit liklihood        
    elif lik=='Gaussian':
        return 1.0/noise_var*(y-m_q)
    elif lik=='Poisson':
        return -np.exp(m_q + 0.5*v_q) + y
    elif lik=='Poisson2':
        f,w = GH_quad(m_q,np.sqrt(v_q))
        y = np.tile(y,[1,f.shape[1]]); term2 = expit(f); t0 = safe_ln(1+safe_exp(f))# avoid dividing by zeros
        d1_log_lik = np.zeros(t0.shape)
        d1_log_lik[t0!=0] = (y[t0!=0]/t0[t0!=0] - 1)*term2[t0!=0]
        return 1.0/_sqrt_2pi*np.dot(  d1_log_lik, w )
    
def calc_lambda(m_q, v_q, y, lik='Poisson', noise_var=0.1):
    if lik=='Bernoulli':
        f,w = GH_quad(m_q,np.sqrt(v_q))
        return 1.0/np.sqrt(2*np.pi)*np.dot(  -expit(y*f)*expit(-y*f), w ) # sigmoid
#         return 1.0/_sqrt_2pi*np.dot( -std_norm_pdf(f)**2 / (std_norm_cdf(y*f)**2+1e-10) - y*f*std_norm_pdf(f) / (std_norm_cdf(y*f)+1e-10), w )
    elif lik=='Gaussian':
        return -1.0/noise_var*np.ones((len(m_q),1))
    elif lik=='Poisson':
        return -np.exp(m_q + 0.5*v_q)
    elif lik=='Poisson2':
        f,w = GH_quad(m_q,np.sqrt(v_q));y = np.tile(y,[1,f.shape[1]]);
        term2 = expit(f)*expit(f); t0 = safe_ln(1+safe_exp(f));d2_log_lik = np.zeros(t0.shape)
        d2_log_lik[t0!=0] = ((y[t0!=0]/t0[t0!=0]-1)*safe_exp(-f[t0!=0])-y[t0!=0]/(t0[t0!=0])**2)*term2[t0!=0]
        return 1.0/_sqrt_2pi*np.dot(  d2_log_lik, w )

def dVLb_dm(m, rho, prior_mean_u, Kmm_inv, A):
    dm = np.dot(A.T,rho) - np.dot(Kmm_inv, m-prior_mean_u) # Eq.(11a) in paper
    return dm

def dVLb_dL(L, lam, Kmm_inv, A, noise_var=1e-8):# optimizing the cholesky factor L guarantees the PSD of V automatically
    dL = np.dot( np.dot( np.dot(A.T, np.diag(lam.ravel())), A ), L ) + inv(L+np.sqrt(noise_var)*np.eye(len(L))).T - np.dot(Kmm_inv, L)
    return dL

In [561]:
def Train(X,Y,ix,X_test=None,y_test=None,max_iter=100,lr=1*10**-5,FPb_cond=1*10**0,stop_cond=1*10**-2,VLB_opt='GD',lik='Poisson'):
    num_train = X.shape[0];num_inducing=len(ix);num_test=X_test.shape[0]; 
    prior_mean_u = np.zeros((num_inducing, 1)); prior_mean_f = np.zeros((num_train, 1))
    Z = X[ix, :]; Z_label = Y[ix] # Z is the inducing set
    model_lap = get_init_hyperparameters(Z,Z_label,lik=lik)#(Z_label+1)/2    
    # variational parameters from initialization
    f_mean, f_var = model_lap._raw_predict(X) 
    length_scale = model_lap.rbf.lengthscale[0]; sigma2 = model_lap.rbf.variance[0];
    if lik=='Gaussian': noise_var=model_lap.Gaussian_noise.variance[0] 
    else: noise_var=1*10**-8
    #  we use variational mean from Laplace appr
    m = f_mean[ix]; V = np.diag(f_var[ix].ravel()); L = cholesky(V)
    Kmm = kernel(Z, Z, l = length_scale, sigma_f = np.sqrt(sigma2)) + noise_var*np.eye(len(Z))
    Kmm_inv = inv(Kmm); Kmn = kernel(Z, X, l = length_scale, sigma_f = np.sqrt(sigma2)); Knn_diag = sigma2*np.ones((num_train, 1))
    A = np.dot(Kmn.T, Kmm_inv) # Knm*inv(Kmm)    
    a = (prior_mean_u,prior_mean_f,A,Kmm,Kmm_inv,Kmn,Knn_diag,Y,noise_var)
    num_iter = 0; VLB = []; VLB_time = []; err = []; FPb_converge = True
    var = np.hstack([m.flatten(), L.flatten()])
    start_time = time.time(); VLB.append(-calc_vlb(m,V, a,lik)[0][0])
    if lik=='Poisson' or lik=='Poisson2':
        y_min = min(Y); y_max = max(Y)
        poisson_lik = predict(Z, X_test, length_scale, sigma2, m, V,Kmm,Kmm_inv,y_min=y_min,y_max=y_max,noise_var=noise_var,lik=lik)
        res = np.argmax(poisson_lik, axis=1)+y_min; err.append( MFE(y_test, res) )
    elif lik=='Gaussian':
        mu_star, std_star = predict(Z, X_test, length_scale, sigma2, m, V,Kmm,Kmm_inv,noise_var=noise_var,lik=lik)
        err.append( 1/num_test*( np.sum( (y_test-mu_star)**2 ) ) )
    elif lik=='Bernoulli':
        p = predict(Z, X_test, length_scale, sigma2, m, V,Kmm,Kmm_inv,noise_var=noise_var,lik=lik); 
        res = np.where(p>=0.5,1,-1)
        err.append( np.sum( np.where(y_test*res<0,1,0) )/num_test );
    VLB_time.append(time.time()-start_time);print('Before iterations, err:{:.6f}'.format(err[-1]) )
    while 1:
        num_iter = num_iter +1;
        m_q = calc_m_q(m, A, prior_mean_u, prior_mean_f); v_q = calc_v_q(V, A, Kmm, Knn_diag);#print('m_q:',m_q[:10])
        # According to Table 1 in paper, expectations of the derivatives wrt N(f|m,v) for Possion likelihood
        rho = calc_rho(m_q, v_q, Y,lik, noise_var); lam = calc_lambda(m_q, v_q,Y,lik, noise_var);        
        if VLB_opt=='GD':
            dm = dVLb_dm(m, rho, prior_mean_u, Kmm_inv, A)
            dL = dVLb_dL(L, lam, Kmm_inv, A, noise_var)
            gradients = np.hstack([dm.flatten(), dL.flatten()])
            var, m_t, v_t = Adam(var,gradients,num_iter,alpha=lr,opt='maximize')
            m = var[:num_inducing][:,None] # variantional mean
            L = var[num_inducing:].reshape(num_inducing,num_inducing) # variational variance V=L*L.T
            V = np.dot(L, L.T);
            
        elif VLB_opt=='FPi':
            dm = dVLb_dm(m, rho, prior_mean_u, Kmm_inv, A)
            m, m_t, v_t = Adam(m,dm,num_iter,alpha=lr,opt='maximize') # print('old V',V[:1,:1])
            V = inv(Kmm_inv-np.dot( np.dot(A.T, np.diag(lam.ravel())), A ));
        elif VLB_opt=='FPb':            
            if FPb_converge:
                dm = dVLb_dm(m, rho, prior_mean_u, Kmm_inv, A); #print('old m:',m[:1]);
                m, m_t, v_t = Adam(m,dm,num_iter,alpha=lr,opt='maximize')
            else:
                V = inv(Kmm_inv-np.dot( np.dot(A.T, np.diag(lam.ravel())), A))
                
        elif VLB_opt=='FPi-mean':
            d = A.T; gamma = -lam; print('old m:',m[:1]);            
            m = np.dot(V, np.dot(d, rho + np.dot(A,m)*gamma) ); 
            V = inv(Kmm_inv-np.dot( np.dot(A.T, np.diag(lam.ravel())), A))

        VLB.append(-calc_vlb(m,V, a,lik)[0][0]); VLB_time.append(time.time()-start_time)
        if lik=='Poisson' or lik=='Poisson2':
            poisson_lik = predict(Z, X_test, length_scale, sigma2, m, V,Kmm,Kmm_inv,y_min=y_min,y_max=y_max,noise_var=noise_var,lik=lik)
            res = np.argmax(poisson_lik, axis=1)+y_min; err.append( MFE(y_test, res) )
        elif lik=='Gaussian':
            mu_star, std_star = predict(Z, X_test, length_scale, sigma2, m, V,Kmm,Kmm_inv,noise_var=noise_var,lik=lik)
            err.append( 1/num_test*( np.sum( (y_test-mu_star)**2 ) ) )
        elif lik=='Bernoulli':
            p = predict(Z, X_test, length_scale, sigma2, m, V,Kmm,Kmm_inv,noise_var=noise_var,lik=lik); res = np.where(p>=0.5,1,-1)
            err.append( np.sum( np.where(y_test*res<0,1,0) )/num_test );
        if num_iter>0:
            delta_vlb = abs(VLB[-1]-VLB[-2])
            if VLB_opt=='FPb' and delta_vlb<=FPb_cond:
                FPb_converge = not FPb_converge; print('m or V converged')
            if num_iter>=5 and delta_vlb<=stop_cond:
                print('After {} iterations it converged: delta_VLB:{:.6f}, VLB:{:.6f}, err:{:.6f}'.format(num_iter,delta_vlb,VLB[-1],err[-1]) );break                
            if num_iter==max_iter:
                print('It has reached the maximum number of iterations, i.e. {}, with delta_VLB:{:.6f}, VLB:{:.6f} and err:{:.6f}'.format(max_iter,delta_vlb,VLB[-1],err[-1]));break
            if num_iter%20 == 0:
                print('iter:{}, delta_VLB:{:.6f}, VLB:{:.6f}, err:{:.6f}'.format(num_iter, delta_vlb, VLB[-1], err[-1]));
            
    return Z,Z_label, VLB, length_scale, sigma2, m, V,noise_var, Kmm, Kmm_inv, A,VLB_time,err

In [ ]:
FPim = Train(X_train,y_train,ix,X_test,y_test,max_iter=4,lr=2*10**-4,VLB_opt='FPi-mean',lik='Gaussian')
Zs,Zs_label, VLB_FPim, length_scale, sigma2, m, V,noise_var, Kmm, Kmm_inv,A,VLB_time_FPim,err_FPim=FPim

In [ ]:
GD = Train(X_train,y_train,ix,X_test,y_test,max_iter=800,lr=4*10**-4,VLB_opt='GD',lik='Bernoulli');
Zs,Zs_label, VLB, length_scale, sigma2, m, V,noise_var, Kmm, Kmm_inv,A,VLB_time,err = GD

In [ ]:
FPb = Train(X_train,y_train,ix,X_test,y_test,max_iter=40,lr=1*10**-4,VLB_opt='FPb',lik='Bernoulli')
Zs,Zs_label, VLB_FPb, length_scale, sigma2, m, V,noise_var, Kmm, Kmm_inv,A,VLB_time_FPb,err_FPb = FPb

In [ ]:
FPi = Train(X_train,y_train,ix,X_test,y_test,max_iter=40,lr=1*10**-4,VLB_opt='FPi',lik='Bernoulli');
Zs,Zs_label, VLB_FPi, length_scale, sigma2, m, V,noise_var, Kmm, Kmm_inv,A,VLB_time_FPi,err_FPi = FPi

In [ ]:
GD2 = Train(X_train,y_train,ix,X_test,y_test,max_iter=400,lr=4*10**-4,VLB_opt='GD',lik='Poisson2');
Zs,Zs_label, VLB2, length_scale, sigma2, m, V,noise_var, Kmm, Kmm_inv,A,VLB_time2,err2 = GD2

In [ ]:
FPb2 = Train(X_train,y_train,ix,X_test,y_test,max_iter=50,lr=2*10**-4,VLB_opt='FPb',lik='Poisson2')
Zs,Zs_label, VLB_FPb2, length_scale, sigma2, m, V,noise_var, Kmm, Kmm_inv,A,VLB_time_FPb2,err_FPb2 = FPb2

In [ ]:
FPi2 = Train(X_train,y_train,ix,X_test,y_test,max_iter=50,lr=2*10**-4,VLB_opt='FPi',lik='Poisson2')
Zs,Zs_label, VLB_FPi2, length_scale, sigma2, m, V,noise_var, Kmm, Kmm_inv,A,VLB_time_FPi2,err_FPi2=FPi2

In [ ]:
FPi = Train(X_train,y_train,ix,X_test,y_test,max_iter=50,lr=2*10**-4,VLB_opt='FPi',lik='Poisson')
Zs,Zs_label, VLB_FPi, length_scale, sigma2, m, V,noise_var, Kmm, Kmm_inv,A,VLB_time_FPi,err_FPi=FPi

In [ ]:
FPi = Train(X_train,y_train,ix,X_test,y_test,max_iter=50,lr=2*10**-4,VLB_opt='FPi',lik='Bernoulli')
Zs,Zs_label, VLB_FPi, length_scale, sigma2, m, V,noise_var, Kmm, Kmm_inv,A,VLB_time_FPi,err_FPi=FPi

In [529]:
%matplotlib

Using matplotlib backend: Qt5Agg


In [544]:
from mpl_toolkits.axes_grid1 import host_subplot
import matplotlib.pyplot as plt
host = host_subplot(111)
par = host.twinx()
host.set_xlabel("Training time(log)")
host.set_ylabel("Error")
par.set_ylabel("negVLB")
# p0, = host.plot(VLB_time, err, 'bo--', lw=1, markersize=5, fillstyle='none', label="Poisson GD error")
# p1, = host.plot(VLB_time2, err2, 'ro--', lw=1, markersize=5, fillstyle='none', label="Poisson2 GD error")
# p2, = par.plot(VLB_time, VLB, 'bo-', lw=1, markersize=5, label="Poisson GD negVLB")
# p3, = par.plot(VLB_time2, VLB2, 'ro-', lw=1, markersize=5, label="Poisson2 GD negVLB")
# p0, = host.plot(VLB_time_FPb, err_FPb, 'bo--', lw=1, markersize=5, fillstyle='none', label="Poisson FPb error")
# p1, = host.plot(VLB_time_FPb2, err_FPb2, 'ro--', lw=1, markersize=5, fillstyle='none', label="Poisson2 FPb error")
# p2, = par.plot(VLB_time_FPb, VLB_FPb, 'bo-', lw=1, markersize=5, label="Poisson FPb negVLB")
# p3, = par.plot(VLB_time_FPb2, VLB_FPb2, 'ro-', lw=1, markersize=5, label="Poisson2 FPb negVLB")
p0, = host.plot(VLB_time_FPi, err_FPi, 'bo--', lw=1, markersize=5, fillstyle='none', label="Poisson FPi error")
p1, = host.plot(VLB_time_FPi2, err_FPi2, 'ro--', lw=1, markersize=5, fillstyle='none', label="Poisson2 FPi error")
p2, = par.plot(VLB_time_FPi, VLB_FPi, 'bo-', lw=1, markersize=5, label="Poisson FPb negVLB")
p3, = par.plot(VLB_time_FPi2, VLB_FPi2, 'ro-', lw=1, markersize=5, label="Poisson2 FPb negVLB")
host.set_xscale('log',basex=10)
leg = plt.legend()
host.yaxis.get_label().set_color(p0.get_color());par.yaxis.get_label().set_color(p0.get_color())
leg.texts[0].set_color(p0.get_color());
leg.texts[1].set_color(p1.get_color())
leg.texts[2].set_color(p2.get_color());
leg.texts[3].set_color(p3.get_color())
plt.show()

In [564]:
from mpl_toolkits.axes_grid1 import host_subplot
import matplotlib.pyplot as plt
host = host_subplot(111)
par = host.twinx()
host.set_xlabel("Training time(log)")
host.set_ylabel("Error")
par.set_ylabel("negVLB")
p0, = host.plot(VLB_time, err, 'bo--', lw=1, markersize=5, fillstyle='none', label="error GD")
p1, = host.plot(VLB_time_FPi, err_FPi, 'ro--', lw=1, markersize=5, fillstyle='none', label="error FPi")
p2, = host.plot(VLB_time_FPb, err_FPb, 'go--', lw=1, markersize=5, fillstyle='none', label="error FPb")
# p3, = host.plot(VLB_time_FPi_mean, err_FPi_mean, 'yo--', markersize=5, lw=1, fillstyle='none', label="FPi-mean")
p3, = par.plot(VLB_time, VLB, 'bo-', lw=1, markersize=5, label="negVLB GD")
p4, = par.plot(VLB_time_FPi, VLB_FPi, 'ro-', lw=1, markersize=5, label="negVLB FPi")
p5, = par.plot(VLB_time_FPb, VLB_FPb, 'go-', lw=1, markersize=5, label="negVLB FPb")
# p7, = par.plot(VLB_time_FPi_mean, VLB_FPi_mean, 'yo-', lw=1, markersize=5, label="FPi-mean")
host.set_xscale('log',basex=10)
leg = plt.legend()
host.yaxis.get_label().set_color(p0.get_color());par.yaxis.get_label().set_color(p0.get_color())
leg.texts[0].set_color(p0.get_color());
leg.texts[1].set_color(p1.get_color())
leg.texts[2].set_color(p2.get_color());
leg.texts[3].set_color(p3.get_color())
leg.texts[4].set_color(p4.get_color());
leg.texts[5].set_color(p5.get_color())
# leg.texts[6].set_color(p6.get_color());
# leg.texts[7].set_color(p7.get_color())
plt.show()

In [492]:
# Make some 1D training data(regression)
num_train = 2000;num_test=700                         # 500 training poitns
X_train = np.linspace(0, 10, num_train)[:,None]       # Inputs evenly spaced between 0 and 10
F = np.sin(X_train)                   # True function (f = sin(x))
y_train = F + 0.01*np.random.randn(num_train)[:,None]  # Observations
X_test = np.linspace(0, 10, num_test)[:,None]       # Inputs evenly spaced between 0 and 10
F_test = np.sin(X_test)                   # True function (f = sin(x))
y_test = F_test + 0.01*np.random.randn(num_test)[:,None]  # Observations
np.random.seed(3);num_inducing=100
ix = random.sample(range(num_train), num_inducing)

In [15]:
y_train = np.where(y_train >= 0, 1, -1); y_test = np.where(y_test >= 0, 1, -1)

In [509]:
# load data(count regression)
mat_contents = sio.loadmat('count_dataset_ucsdpeds1l_N4000_D30.mat')
x = mat_contents['x']; y = mat_contents['y']
x = x.astype('double')

# shuffle the data and split data into training set and test set
data = np.concatenate((x,y), axis=1)
np.random.shuffle(data)
num_data = data.shape[0]; dim_data = data.shape[1] - 1;
num_train = int(0.5*np.ceil(num_data)); num_test = num_data - num_train
x_train = data[:num_train, :-1]; y_train = data[:num_train, -1];
x_test = data[num_train:, :-1]; y_test = data[num_train:, -1];
y_train = y_train[:, None]; y_test = y_test[:, None]
# data Standardization with zero mean and unit variance
scaler = preprocessing.StandardScaler().fit(x_train)
X_train = scaler.transform(x_train); X_test = scaler.transform(x_test)
np.random.seed(3);num_inducing=100
ix = random.sample(range(num_train), num_inducing)

In [546]:
# Load data from file, make sure banana.csv is in the same directory as this notebook
data = np.genfromtxt('banana.csv', delimiter=',')

# Dimension of data
D = data.shape[1]-1

# Seperate our data (input) from its corresponding label output
# .. note we have to rescale from [-1,1] to [0,1] for a Bernoulli distribution
X, y = data[:,:D], data[:,-1][:, None]

# We will plot our data as well
plt.figure(figsize=(8,8))

# Plot 0 class in blue
plt.plot(X[np.where(y == -1),0],X[np.where(y == -1),1],'bo', mew=0.5, alpha=0.5)
# Plot 1 class in red
plt.plot(X[np.where(y == 1),0],X[np.where(y == 1),1],'ro', mew=0.5, alpha=0.5)

# Annotate plot
plt.xlabel("$x_1$"), plt.ylabel("$x_2$")
plt.title("Banana Dataset (red=1, blue=0)")
plt.axis("square"), plt.xlim((-3, 3)), plt.ylim((-3, 3));

In [547]:
# Load data from file, make sure banana.csv is in the same directory as this notebook
data = np.genfromtxt('banana.csv', delimiter=',')

# Dimension of data
D = data.shape[1]-1
n = data.shape[0]
# Seperate our data (input) from its corresponding label output
# .. note we have to rescale from [-1,1] to [0,1] for a Bernoulli distribution
X, y = data[:,:D], data[:,-1][:, None]
num_train = 3000; num_test = n - num_train
X_train, y_train = X[:num_train,:], y[:num_train]
X_test, y_test = X[:num_test,:], y[:num_test]
np.random.seed(3);num_inducing=100 # randomly select active set and then keep them fixed
ix = random.sample(range(num_train), num_inducing)

In [ ]:
from scipy.stats import norm
num = np.arange(-39,40);print(num)
plt.figure(1)
a1 = norm.cdf(num);print(a1)
a2 = norm.pdf(num);print(a2)
plt.plot(a1)
plt.plot(a2)
plt.figure(2)
r = norm.pdf(num)/norm.cdf(num)
plt.plot(r);print(r.shape)
print(r)

In [463]:
norm.pdf(-30)

1.4736461348785476e-196

In [ ]:
aa=np.random.randn(4,2);cc=np.ones((4,2))*10;print(aa)
bb=np.arange(4)[:,None];print(bb)
bb=np.tile(bb,[1,2]);print(bb.shape)
term1=np.zeros((4,2))
term1[bb!=0]=(aa[bb!=0]/cc[bb!=0])
print(term1)
# a1=np.where(bb==0,0,term1);print('a1',a1)

In [ ]:
np.all(np.linalg.eigvals(V) > 0)# check PSD condition for V

In [ ]:
result = {
  'a': lambda x: x * 5,
  'b': lambda x: x + 7,
  'c': lambda x: x - 2
}[value](x)